### Домашнее задание №6.

1. Взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php).
2. Обучить любой классификатор (какой вам нравится).
3. Разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные примеры (класс 1), а только лишь часть.
4. Применить random negative sampling для построения классификатора в новых условиях.
5. Сравнить качество с решением из пункта 3 (построить отчет - таблицу метрик).
6. *Поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P).

Возьмем датасет бинарной классификации, представляющий собой электронную медицинискую карту пациентов с результатами лабораторных анализов, для определения дальнейшего хода лечения (требуется дальнейший уход за пациентом или нет).

In [1]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, precision_recall_curve

In [2]:
df = pd.read_csv('data-ori.csv')

In [3]:
df.head()

,HAEMATOCRIT,HAEMOGLOBINS,ERYTHROCYTE,LEUCOCYTE,THROMBOCYTE,MCH,MCHC,MCV,AGE,SEX,SOURCE
0,35.1,11.8,4.65,6.3,310,25.4,33.6,75.5,1,F,out
1,43.5,14.8,5.39,12.7,334,27.5,34.0,80.7,1,F,out
2,33.5,11.3,4.74,13.2,305,23.8,33.7,70.7,1,F,out
3,39.1,13.7,4.98,10.5,366,27.5,35.0,78.5,1,F,out
4,30.9,9.9,4.23,22.1,333,23.4,32.0,73.0,1,M,out


In [4]:
df.describe()

,HAEMATOCRIT,HAEMOGLOBINS,ERYTHROCYTE,LEUCOCYTE,THROMBOCYTE,MCH,MCHC,MCV,AGE
count,4412.000000,4412.000000,4412.000000,4412.000000,4412.000000,4412.000000,4412.000000,4412.000000,4412.000000
mean,38.197688,12.741727,4.541260,8.718608,257.524479,28.234701,33.343042,84.612942,46.626473
std,5.974784,2.079903,0.784091,5.049041,113.972365,2.672639,1.228664,6.859101,21.731218
min,13.700000,3.800000,1.480000,1.100000,8.000000,14.900000,26.000000,54.000000,1.000000
25%,34.375000,11.400000,4.040000,5.675000,188.000000,27.200000,32.700000,81.500000,29.000000
50%,38.600000,12.900000,4.570000,7.600000,256.000000,28.700000,33.400000,85.400000,47.000000
75%,42.500000,14.200000,5.050000,10.300000,321.000000,29.800000,34.100000,88.700000,64.000000
max,69.000000,18.900000,7.860000,76.600000,1183.000000,40.800000,39.000000,115.600000,99.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4412 entries, 0 to 4411
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HAEMATOCRIT   4412 non-null   float64
 1   HAEMOGLOBINS  4412 non-null   float64
 2   ERYTHROCYTE   4412 non-null   float64
 3   LEUCOCYTE     4412 non-null   float64
 4   THROMBOCYTE   4412 non-null   int64  
 5   MCH           4412 non-null   float64
 6   MCHC          4412 non-null   float64
 7   MCV           4412 non-null   float64
 8   AGE           4412 non-null   int64  
 9   SEX           4412 non-null   object 
 10  SOURCE        4412 non-null   object 
dtypes: float64(7), int64(2), object(2)
memory usage: 379.3+ KB


In [6]:
df['SOURCE'].value_counts()

out    2628
in     1784
Name: SOURCE, dtype: int64

- Соберем пайплайн для признаков.

In [7]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        return pd.get_dummies(X, prefix=self.key)[self.columns]

In [8]:
transformers = []

for cont_col in df.columns.drop(['SEX', 'SOURCE']):
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    transformers.append((cont_col, cont_transformer))

transformers.append(('SEX', Pipeline([('selector', FeatureSelector(column='SEX')),
                                         ('ohe', OHEEncoder(key='SEX'))])
                    ))

feats = FeatureUnion(transformers)

- Напишем функцию для обучения модели и подсчета метрик классификации.

In [9]:
def learn_model_eval_results(model, X_train, y_train, X_test, y_test, verbose=False):
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_test)[:, 1]
    
    prc, rec, ths = precision_recall_curve(y_test, preds)
    prc[(prc == 0) & (rec == 0)] = np.e-10
    f1 = (2 * prc * rec) / (prc + rec)
    
    ix = np.argmax(f1)
    
    if verbose:
        print('Classification results:')
        print(f"f1: {f1[ix] * 100.0:.2f}%") 
        print(f"recall: {rec[ix] * 100.0:.2f}%") 
        print(f"precision: {prc[ix] * 100.0:.2f}%" )
        roc = roc_auc_score(y_test, preds)
        print(f"roc: {roc * 100.0:.2f}%")
    
    return f1[ix], prc[ix], rec[ix], ths[ix], roc

- Заменим значения целевого признака на 0 и 1. Далее разобьём датасет на train и test.

In [10]:
df['SOURCE'] = np.where(df['SOURCE'] == 'in', 1, 0).astype('i1')

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['SOURCE'], axis=1), df['SOURCE'], random_state=23)

- Построим, обучим модель бустинга и выполним предсказания. Посчитаем и запомним метрики классификации.

In [12]:
xgb = Pipeline([('features', feats),
                ('classifier', XGBClassifier(
                    n_estimators=100,
                    learning_rate=0.1,
                    max_depth=5,
                    use_label_encoder=False,
                    eval_metric='error',
                    random_state=23))])

pivot_metric_table = []

In [13]:
pivot_metric_table.append(learn_model_eval_results(xgb, X_train, y_train, X_test, y_test, verbose=True))

Classification results:
f1: 68.86%
recall: 82.66%
precision: 59.00%
roc: 80.42%


- Для перевода представдения датасета как Positive Unlabeled напишем пару функций. Первая добавляет целевой признак с заданной долей позитивного класса, вторая делит датасет на обучающий (в нем все объекты с позитивным классом и столько же неразмеченных объектов) и тестовый (в нём оставшиеся объекты).

In [14]:
def add_pu_column(df, target, perc):
    mod_df = df.copy()
    
    pos_ind = np.where(mod_df[target] == 1)[0]
    np.random.shuffle(pos_ind)
    
    pos_sample = pos_ind[:int(np.ceil(perc * len(pos_ind)))]
    
    mod_df['pu_col'] = 0
    mod_df.loc[pos_sample,'pu_col'] = 1
    
    return mod_df

def train_test_sample_split(df, real_target, pu_target):
    df = df.copy()
    
    pos_df = df[df[pu_target] == 1]
    neg_df = df[df[pu_target] == 0]
    
    pos_sample = pos_df.copy()
    neg_sample = neg_df[:pos_df.shape[0]]
    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
    sample_test = neg_df[pos_df.shape[0]:]
    
    return sample_train.drop([real_target, pu_target], axis=1), \
           sample_test.drop([real_target, pu_target], axis=1), \
           sample_train[real_target], \
           sample_test[real_target], \
           sample_train[pu_target], \
           sample_test[pu_target]

- Обучим модели на датасетах с заданными долями позитивным классов и посчитаем метрики.

In [15]:
for perc in [0.1, 0.2, 0.3, 0.3, 0.4]:
    mod_df = add_pu_column(df, 'SOURCE', perc)
    X_train, X_test, y_train, y_test, y_train_pu, y_test_pu = train_test_sample_split(mod_df, 'SOURCE', 'pu_col')
    print(f'{perc * 100} % of positive labels:')
    pivot_metric_table.append(learn_model_eval_results(xgb, X_train, y_train_pu, X_test, y_test, verbose=True))
    print(30 * '*')

10.0 % of positive labels:
Classification results:
f1: 55.62%
recall: 99.87%
precision: 38.54%
roc: 60.12%
******************************
20.0 % of positive labels:
Classification results:
f1: 54.87%
recall: 80.11%
precision: 41.72%
roc: 61.74%
******************************
30.0 % of positive labels:
Classification results:
f1: 51.76%
recall: 70.17%
precision: 41.00%
roc: 64.51%
******************************
30.0 % of positive labels:
Classification results:
f1: 50.53%
recall: 95.20%
precision: 34.39%
roc: 58.89%
******************************
40.0 % of positive labels:
Classification results:
f1: 52.59%
recall: 69.57%
precision: 42.28%
roc: 67.71%
******************************


- Сведем метрики по всем моделям в одну таблицу.

In [16]:
pivot_metric_table = pd.DataFrame(pivot_metric_table)
pivot_metric_table.columns = ['F score', 'Precision', 'Recall', 'Optimal threshold', 'ROC AUC score']
pivot_metric_table['Classifier'] = ['xgb', 'xgb_pu_10', 'xgb_pu_20', 'xgb_pu_30', 'xgb_pu_40', 'xgb_pu_50']
pivot_metric_table.set_index('Classifier', inplace=True)

In [17]:
pivot_metric_table

,F score,Precision,Recall,Optimal threshold,ROC AUC score
Classifier,,,,,
xgb,0.688555,0.590032,0.826577,0.258112,0.804208
xgb_pu_10,0.556211,0.385435,0.998716,0.987335,0.601185
xgb_pu_20,0.548682,0.417229,0.801067,0.996794,0.617387
xgb_pu_30,0.517553,0.409958,0.701723,0.998251,0.645112
xgb_pu_40,0.505283,0.343903,0.952036,0.996408,0.588939
xgb_pu_50,0.525917,0.422764,0.695652,0.998157,0.677057
